Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
#first follow the instructions in the README.md file under Local Installation
!pip3 install -r requirements.txt
# !python3 setup.py install

In [1]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
model_path = "models/19200_gpt.pth"
tts = TextToSpeech(model_path=model_path, use_deepspeed=True, kv_cache=True)

/home/ansel/.pyenv/versions/3.12.7/envs/tortoise/lib/python3.12/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/ansel/.pyenv/versions/3.12.7/envs/tortoise/lib/python3.12/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/ansel/.pyenv/versions/3.12.7/envs/tortoise/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/hom

[2024-10-09 13:30:55,836] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2024-10-09 13:30:56,603] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.1, git-hash=unknown, git-branch=unknown
[2024-10-09 13:30:56,603] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-10-09 13:30:56,604] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Using /home/ansel/.cache/torch_extensions/py312_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ansel/.cache/torch_extensions/py312_cu121/transformer_inference/build.ninja...
/home/ansel/.pyenv/versions/3.12.7/envs/tortoise/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load transformer_inference op: 0.050508737564086914 seconds
[2024-10-09 13:30:56,682] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode'

Loading extension module transformer_inference...
/home/ansel/dev/tortoise-tts/tortoise/api.py:332: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(get_model_path("

In [2]:
# This is the text that will be spoken.
text = "Hey, it's John Roderick. You might know me from all the great shows!"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "ultra_fast"

In [3]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/john/1.wav')

angie/                geralt/  pat/           train_daws/      train_mouse/
applejack/            halle/   pat2/          train_dotrice/   weaver/
cond_latent_example/  jlaw/    rainbow/       train_dreams/    william/
daniel/               john/    snakes/        train_empire/
deniro/               lj/      tim_reynolds/  train_grace/
emma/                 mol/     tom/           train_kennard/
freeman/              myself/  train_atkins/  train_lescault/


In [10]:
# Pick one of the voices from the output above
voice = "john"

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(
    text,
    voice_samples=voice_samples,
    conditioning_latents=conditioning_latents,
    preset=preset,
)
torchaudio.save("generated.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("generated.wav")

Generating autoregressive samples..


100%|██████████| 1/1 [00:00<00:00,  2.04it/s]


Computing best candidates using CLVP


100%|██████████| 1/1 [00:00<00:00, 32.77it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:00<00:00, 118.62it/s]


In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(
    text, voice_samples=None, conditioning_latents=None, preset=preset
)
torchaudio.save("generated.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("generated.wav")

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(["pat", "william"])

gen = tts.tts_with_preset(
    "They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.",
    voice_samples=None,
    conditioning_latents=None,
    preset=preset,
)
torchaudio.save("captain_kirkard.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("captain_kirkard.wav")

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..